# Practical Lab: Week 6

Note: This practical session is based on the exercises in chapter 4, section 4.7.2 and Applied 13.

# Recursos : 

https://colab.research.google.com/github/intro-stat-learning/ISLP_labs/blob/v2/Ch04-classification-lab.ipynb#scrollTo=78b2946b

First, let's install the packages that we will use in this lab

For these exercises we use the numpy library in version 1.22, which is compatible with the ISLP library.

In [1]:
!pip install --force-reinstall -v "numpy==1.22"
!pip install ISLP

Traceback (most recent call last):
  File "/usr/bin/pip", line 33, in <module>
    sys.exit(load_entry_point('pip==20.3.4', 'console_scripts', 'pip')())
  File "/usr/bin/pip", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/usr/lib/python3.9/importlib/metadata.py", line 77, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 790, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/mai

If you are working with Google Colab, after running this cell, restart the terminal.

In [3]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from ISLP import load_data
from ISLP import confusion_table

## Part 1

In this part of the lab we will work with the Smarket dataframe. (https://islp.readthedocs.io/en/latest/datasets/Smarket.html)

In [43]:
Smarket = load_data('Smarket')
Smarket_load = load_data('Smarket')
Smarket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       1250 non-null   int64  
 1   Lag1       1250 non-null   float64
 2   Lag2       1250 non-null   float64
 3   Lag3       1250 non-null   float64
 4   Lag4       1250 non-null   float64
 5   Lag5       1250 non-null   float64
 6   Volume     1250 non-null   float64
 7   Today      1250 non-null   float64
 8   Direction  1250 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 88.0+ KB


1 - Compute the correlation matrix using the `corr()` method for data `Smarket`. Which pair of variables have substantial correlation? Why `direction` not appear?

In [7]:
Smarket.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [6]:
Smarket.corr()

/tmp/ipykernel_13423/1907124636.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Smarket.corr()


,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
Year,1.000000,0.029700,0.030596,0.033195,0.035689,0.029788,0.539006,0.030095
Lag1,0.029700,1.000000,-0.026294,-0.010803,-0.002986,-0.005675,0.040910,-0.026155
Lag2,0.030596,-0.026294,1.000000,-0.025897,-0.010854,-0.003558,-0.043383,-0.010250
Lag3,0.033195,-0.010803,-0.025897,1.000000,-0.024051,-0.018808,-0.041824,-0.002448
Lag4,0.035689,-0.002986,-0.010854,-0.024051,1.000000,-0.027084,-0.048414,-0.006900
Lag5,0.029788,-0.005675,-0.003558,-0.018808,-0.027084,1.000000,-0.022002,-0.034860
Volume,0.539006,0.040910,-0.043383,-0.041824,-0.048414,-0.022002,1.000000,0.014592
Today,0.030095,-0.026155,-0.010250,-0.002448,-0.006900,-0.034860,0.014592,1.000000


2 - Fit a logistic regression model in order to predict Direction using Lag1 through Lag5 and Volume.

# Metodo 1
https://www.datacamp.com/tutorial/understanding-logistic-regression-python

In [8]:
#split dataset in features and target variable
feature_cols = ['Lag1', 'Lag2', 'Lag3', 'Lag4','Lag5']
X = Smarket[feature_cols] # Features
y = Smarket.Direction # Target variable

In [12]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X, y)

y_pred = logreg.predict(X)
print(y_pred)

['Up' 'Down' 'Down' ... 'Up' 'Up' 'Up']


# Metodo 2
https://www.andrewvillazon.com/logistic-regression-python-statsmodels/

In [80]:
Smarket = pd.get_dummies(Smarket, columns = ['Direction'])

In [81]:
log_reg = smf.logit("Direction_Up ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume", data=Smarket).fit()
print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.691034
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:           Direction_Up   No. Observations:                 1250
Model:                          Logit   Df Residuals:                     1243
Method:                           MLE   Df Model:                            6
Date:                Mon, 13 Nov 2023   Pseudo R-squ.:                0.002074
Time:                        22:09:33   Log-Likelihood:                -863.79
converged:                       True   LL-Null:                       -865.59
Covariance Type:            nonrobust   LLR p-value:                    0.7319
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1260      0.241     -0.523      0.601      -0.598       0.346
Lag1          -0.0731      0.

3 - Use the `pvalues` attribute to access the p-values for the coefficients (not shown).

In [82]:
log_reg.pvalues

Intercept    0.600700
Lag1         0.145232
Lag2         0.398352
Lag3         0.824334
Lag4         0.851445
Lag5         0.834998
Volume       0.392404
dtype: float64

4 - Use `predict()` method to predict the probability that the market will go up, given values of the predictors.

In [83]:
ypred = log_reg.predict()
ypred

array([0.50708413, 0.48146788, 0.48113883, ..., 0.5392683 , 0.52611829,
       0.51791656])

5 - Create a array with "Up" and "Down" for all probabilities calculated before. Consider  prob > 0.5 "Up"

In [84]:
# if prob > 0.5 -> "Up"

## Convert ypred column to up/down

ypred_labels = np.array(['Down']*len(Smarket))
ypred_labels[ypred > 0.5] = "Up"
ypred_labels 

array(['Up', 'Down', 'Down', ..., 'Up', 'Up', 'Up'], dtype='<U4')

In [85]:
confusion_table_result = confusion_table(ypred_labels , Smarket_load.Direction)
confusion_table_result

Truth,Down,Up
Predicted,,
Down,145,141
Up,457,507


6 - The `confusion_table()` function from the ISLP package summarizes these predictions, showing how many observations were correctly or incorrectly classified. Compare using `confusion_table` the labels of predictions and the column Direction from dataset. The diagonal elements of the confusion matrix indicate correct predictions, while the off-diagonals represent incorrect predictions.

In [87]:
np.mean(ypred_labels == Smarket_load.Direction) # (116 + 550)/1250

0.5216

In [55]:
resss = "\n".join("{} {}".format(x, y) for x, y in zip(ypred_labels, Smarket_load.Direction))
print(resss)

Up Up
Down Up
Down Down
Up Up
Up Up
Up Up
Down Down
Up Up
Up Up
Down Up
Up Down
Up Down
Up Up
Up Up
Down Down
Up Up
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Up
Up Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Down
Up Up
Down Down
Up Down
Up Up
Up Down
Up Up
Up Up
Down Up
Down Up
Up Down
Up Down
Up Up
Up Down
Up Up
Up Down
Up Up
Up Down
Up Down
Up Down
Up Up
Down Up
Down Up
Down Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Up
Down Down
Down Up
Up Up
Down Down
Up Up
Down Down
Up Up
Up Up
Down Up
Down Down
Up Down
Up Down
Up Up
Up Up
Down Up
Down Down
Up Up
Up Up
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Down
Up Up
Up Up
Up Up
Down Up
Down Up
Up Up
Down Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Up
Up Up
Down Up
Up Up
Down Down
Up Up
Up Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Up
Up Up
Down Up
Down Down
Up Down
Up Down
Up Down
Up Up
Down Down
Up Up
Up Down
Up Down
Up Down
Up Up
Up Down
Up Down
Up Up
Down Up
Down Down
Up Up
Up Down
Up Up
Up Down
Up D

## Part 2

This part of lab should be answered using the Weekly data set, which is part of the ISLP package. This data is similar in nature to the Smarket data from previous lab, except that it contains 1,089 weekly returns for 21 years, from the beginning of 1990 to the end of 2010.
(https://islp.readthedocs.io/en/latest/datasets/Weekly.html)

In [69]:
Weekly = load_data('Weekly')
Weekly_load = load_data('Weekly')

Weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       1089 non-null   int64  
 1   Lag1       1089 non-null   float64
 2   Lag2       1089 non-null   float64
 3   Lag3       1089 non-null   float64
 4   Lag4       1089 non-null   float64
 5   Lag5       1089 non-null   float64
 6   Volume     1089 non-null   float64
 7   Today      1089 non-null   float64
 8   Direction  1089 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 76.7+ KB


1 - Produce some numerical and graphical summaries of the Weekly data. Do there appear to be any patterns?

In [62]:
Weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [70]:
Weekly = pd.get_dummies(Weekly, columns = ['Direction'])
# remove up/down from direction, substitute with Direction_Down and Direction_Up columns with 0/1 values

In [71]:
Weekly

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction_Down,Direction_Up
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,1,0
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,1,0
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,0,1
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,0,1
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,0,1
...,...,...,...,...,...,...,...,...,...,...
1084,2010,-0.861,0.043,-2.173,3.599,0.015,3.205160,2.969,0,1
1085,2010,2.969,-0.861,0.043,-2.173,3.599,4.242568,1.281,0,1
1086,2010,1.281,2.969,-0.861,0.043,-2.173,4.835082,0.283,0,1
1087,2010,0.283,1.281,2.969,-0.861,0.043,4.454044,1.034,0,1


2 - Use the full data set to perform a logistic regression with `Direction` as the response and the five lag variables plus Volume as predictors. Use the summary function to print the results. Do any of the predictors appear to be statistically significant? If so, which ones?

In [72]:
log_reg_weekly = smf.logit("Direction_Up ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume", data=Weekly).fit()
print(log_reg_weekly.summary())

Optimization terminated successfully.
         Current function value: 0.682441
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:           Direction_Up   No. Observations:                 1089
Model:                          Logit   Df Residuals:                     1082
Method:                           MLE   Df Model:                            6
Date:                Mon, 13 Nov 2023   Pseudo R-squ.:                0.006580
Time:                        22:00:34   Log-Likelihood:                -743.18
converged:                       True   LL-Null:                       -748.10
Covariance Type:            nonrobust   LLR p-value:                    0.1313
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2669      0.086      3.106      0.002       0.098       0.435
Lag1          -0.0413      0.

##### Reflection 

##### statistically significant predictos?



In [73]:
ypred_direction = log_reg_weekly.predict()
ypred_direction

array([0.60862494, 0.60103144, 0.58756995, ..., 0.57972297, 0.55091703,
       0.52212163])

3 - Compute the confusion matrix and overall fraction of correct predictions. Explain what the confusion matrix is telling you about the types of mistakes made by logistic regression.

In [74]:
# if prob > 0.5 -> "Up"

## Convert ypred column to up/down

ypred_direction_labels = np.array(['Down']*len(Weekly))
ypred_direction_labels[ypred_direction > 0.5] = "Up"
ypred_direction_labels 

array(['Up', 'Up', 'Up', ..., 'Up', 'Up', 'Up'], dtype='<U4')

In [75]:
confusion_table_result = confusion_table(ypred_direction_labels , Weekly_load.Direction)
confusion_table_result

Truth,Down,Up
Predicted,,
Down,54,48
Up,430,557


In [77]:
np.mean(ypred_direction_labels == Weekly_load.Direction) # (116 + 550)/1250

0.5610651974288338

In [78]:
rezzz = "\n".join("{} {}".format(x, y) for x, y in zip(ypred_labels, Smarket_load.Direction))
print(rezzz)

Up Up
Down Up
Down Down
Up Up
Up Up
Up Up
Down Down
Up Up
Up Up
Down Up
Up Down
Up Down
Up Up
Up Up
Down Down
Up Up
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Up
Up Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Down
Up Up
Down Down
Up Down
Up Up
Up Down
Up Up
Up Up
Down Up
Down Up
Up Down
Up Down
Up Up
Up Down
Up Up
Up Down
Up Up
Up Down
Up Down
Up Down
Up Up
Down Up
Down Up
Down Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Up
Down Down
Down Up
Up Up
Down Down
Up Up
Down Down
Up Up
Up Up
Down Up
Down Down
Up Down
Up Down
Up Up
Up Up
Down Up
Down Down
Up Up
Up Up
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Down
Up Up
Up Up
Up Up
Down Up
Down Up
Up Up
Down Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Up
Up Up
Down Up
Up Up
Down Down
Up Up
Up Down
Up Down
Up Up
Up Down
Up Down
Up Down
Up Down
Up Up
Up Up
Down Up
Down Down
Up Down
Up Down
Up Down
Up Up
Down Down
Up Up
Up Down
Up Down
Up Down
Up Up
Up Down
Up Down
Up Up
Down Up
Down Down
Up Up
Up Down
Up Up
Up Down
Up D

In [89]:
for pred,real in zip(ypred_labels, Smarket_load.Direction):
    print(f"real = {real} | previsto={pred} => {real==pred}")

real = Up | previsto=Up => True
real = Up | previsto=Down => False
real = Down | previsto=Down => True
real = Up | previsto=Up => True
real = Up | previsto=Up => True
real = Up | previsto=Up => True
real = Down | previsto=Down => True
real = Up | previsto=Up => True
real = Up | previsto=Up => True
real = Up | previsto=Down => False
real = Down | previsto=Down => True
real = Down | previsto=Up => False
real = Up | previsto=Up => True
real = Up | previsto=Down => False
real = Down | previsto=Down => True
real = Up | previsto=Up => True
real = Down | previsto=Up => False
real = Up | previsto=Up => True
real = Down | previsto=Up => False
real = Down | previsto=Down => True
real = Down | previsto=Up => False
real = Down | previsto=Up => False
real = Up | previsto=Up => True
real = Down | previsto=Down => True
real = Down | previsto=Up => False
real = Up | previsto=Up => True
real = Down | previsto=Down => True
real = Down | previsto=Up => False
real = Down | previsto=Up => False
real = Down

4 - Now fit the logistic regression model using a training data period from 1990 to 2008, with Lag2 as the only predictor. Compute the confusion matrix and the overall fraction of correct predictions for the held out data (that is, the data from 2009 and 2010).

http://www.science.smith.edu/~jcrouser/SDS293/labs/lab4-py.html